#### importing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/Shared drives/Adversarial NLP"

Mounted at /content/drive
/content/drive/Shared drives/Adversarial NLP


In [ ]:
!pip install transformers
!pip install textattack
!pip install huggingface_hub
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 17.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 KB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 KB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import itertools
import numpy as np
import pandas as pd
import math
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
import textattack
import transformers
from transformers import AutoTokenizer, RobertaModel, RobertaConfig, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments, AutoModel, AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import PreTrainedTokenizer
import torch
import evaluate

from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack_recipes import TextFoolerJin2019
from textattack.models.wrappers import ModelWrapper

import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaModel, RobertaTokenizer, RobertaForSequenceClassification
import torch.optim as optim
from torch.utils.data import DataLoader


import random

from textattack.models.helpers import T5ForTextToText
from textattack.models.tokenizers import T5Tokenizer

from textattack.models.wrappers.pytorch_model_wrapper import PyTorchModelWrapper
# Load the AG News Dataset
train = load_dataset('ag_news', split='train')
train = train['text']
test = load_dataset('ag_news', split='test')
test = test['text']

# Define the model and tokenizer
model_name = "textattack/roberta-base-ag-news"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


Some weights of the model checkpoint at textattack/roberta-base-ag-news were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Masking based on attack mechanism

In [ ]:
torch.cuda.empty_cache()

# HuggingFaceModelWrapper
class MyWrapperVulnerableWordMask(PyTorchModelWrapper):
    """Loads a HuggingFace ``transformers`` model and tokenizer."""

    def __init__(self, model, tokenizer):
        assert isinstance(
            model, (transformers.PreTrainedModel, T5ForTextToText)
        ), f"`model` must be of type `transformers.PreTrainedModel`, but got type {type(model)}."
        assert isinstance(
            tokenizer,
            (
                transformers.PreTrainedTokenizer,
                transformers.PreTrainedTokenizerFast,
                T5Tokenizer,
            ),
        ), f"`tokenizer` must of type `transformers.PreTrainedTokenizer` or `transformers.PreTrainedTokenizerFast`, but got type {type(tokenizer)}."

        self.model = model
        self.tokenizer = tokenizer
        # self.tfidf_vocab = tfidf_vocab

    def __call__(self, text_input_list):
        """Passes inputs to HuggingFace models as keyword arguments.
        (Regular PyTorch ``nn.Module`` models typically take inputs as
        positional arguments.)
        """
        model_device = next(self.model.parameters()).device
#############################
        # from BERT attack github
        filter_words = ['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'ain', 'all', 'almost',
                        'alone', 'along', 'already', 'also', 'although', 'am', 'among', 'amongst', 'an', 'and', 'another',
                        'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'aren', "aren't", 'around', 'as',
                        'at', 'back', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides',
                        'between', 'beyond', 'both', 'but', 'by', 'can', 'cannot', 'could', 'couldn', "couldn't", 'd', 'didn',
                        "didn't", 'doesn', "doesn't", 'don', "don't", 'down', 'due', 'during', 'either', 'else', 'elsewhere',
                        'empty', 'enough', 'even', 'ever', 'everyone', 'everything', 'everywhere', 'except', 'first', 'for',
                        'former', 'formerly', 'from', 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'he', 'hence',
                        'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his',
                        'how', 'however', 'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'isn', "isn't", 'it', "it's",
                        'its', 'itself', 'just', 'latter', 'latterly', 'least', 'll', 'may', 'me', 'meanwhile', 'mightn',
                        "mightn't", 'mine', 'more', 'moreover', 'most', 'mostly', 'must', 'mustn', "mustn't", 'my', 'myself',
                        'namely', 'needn', "needn't", 'neither', 'never', 'nevertheless', 'next', 'no', 'nobody', 'none',
                        'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'o', 'of', 'off', 'on', 'once', 'one', 'only',
                        'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'per',
                        'please', 's', 'same', 'shan', "shan't", 'she', "she's", "should've", 'shouldn', "shouldn't", 'somehow',
                        'something', 'sometime', 'somewhere', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs',
                        'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein',
                        'thereupon', 'these', 'they', 'this', 'those', 'through', 'throughout', 'thru', 'thus', 'to', 'too',
                        'toward', 'towards', 'under', 'unless', 'until', 'up', 'upon', 'used', 've', 'was', 'wasn', "wasn't",
                        'we', 'were', 'weren', "weren't", 'what', 'whatever', 'when', 'whence', 'whenever', 'where',
                        'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while',
                        'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'with', 'within', 'without', 'won',
                        "won't", 'would', 'wouldn', "wouldn't", 'y', 'yet', 'you', "you'd", "you'll", "you're", "you've",
                        'your', 'yours', 'yourself', 'yourselves']
       
        percent_to_mask = 20
        for i in range(len(text_input_list)):
            sentence = text_input_list[i]
            words = sentence.split()
            combinations = []
            for j in range(len(words)):
                masked_sentence = " ".join(["<mask>" if k == j else words[k] for k in range(len(words))])
                combinations.append(masked_sentence)
            
            scores = []
            with torch.no_grad():
                original_logit = self.model(**self.tokenizer(sentence, add_special_tokens=True, max_length=128, padding=True, truncation=True, return_tensors="pt").to(model_device))[0]
            for combination in combinations:
                with torch.no_grad():
                    logit = self.model(**self.tokenizer(combination, add_special_tokens=True, max_length=128, padding=True, truncation=True, return_tensors="pt").to(model_device))[0]
                    # get the euclidean distance between original logit and the result
                    score = torch.dist(original_logit, logit)
                scores.append(score)
            
            # make a tuple of words : importance scores
            word_importance = [(word, score) for word, score in zip(words, scores)]

            # get the n most important words
            filtered_list = [pair for pair in word_importance if pair[0] not in filter_words]
    
            sorted_list = sorted(filtered_list, key=lambda x: x[1], reverse=True)
                        
            num_to_mask = int(len(words) * percent_to_mask / 100)

            top_n_words = [word for word, _ in sorted_list[:num_to_mask]]

            # top_words = sorted(word_importance, key=lambda x: x[1], reverse=True)[:num_to_mask]

            # replace the n most important words with "<mask>"
            text_input_list[i] = " ".join(["<mask>" if word in [x[0] for x in top_n_words] else word for word in words])

#############################
        # Default max length is set to be int(1e30), so we force 512 to enable batching.
        max_length = (
            512
            if self.tokenizer.model_max_length == int(1e30)
            else self.tokenizer.model_max_length
        )
        inputs_dict = self.tokenizer(
            text_input_list,
            add_special_tokens=True,
            padding="max_length",
            max_length=max_length,
            truncation=True,
            return_tensors="pt",
        )

       
        
        inputs_dict.to(model_device)

        with torch.no_grad():
            outputs = self.model(**inputs_dict)

        if isinstance(outputs[0], str):
            # HuggingFace sequence-to-sequence models return a list of
            # string predictions as output. In this case, return the full
            # list of outputs.
            return outputs
        else:
            # HuggingFace classification models return a tuple as output
            # where the first item in the tuple corresponds to the list of
            # scores for each input.
            return outputs.logits

    def get_grad(self, text_input):
        """Get gradient of loss with respect to input tokens.
        Args:
            text_input (str): input string
        Returns:
            Dict of ids, tokens, and gradient as numpy array.
        """
        if isinstance(self.model, textattack.models.helpers.T5ForTextToText):
            raise NotImplementedError(
                "`get_grads` for T5FotTextToText has not been implemented yet."
            )

        self.model.train()
        embedding_layer = self.model.get_input_embeddings()
        original_state = embedding_layer.weight.requires_grad
        embedding_layer.weight.requires_grad = True

        emb_grads = []

        def grad_hook(module, grad_in, grad_out):
            emb_grads.append(grad_out[0])

        emb_hook = embedding_layer.register_backward_hook(grad_hook)

        self.model.zero_grad()
        model_device = next(self.model.parameters()).device
        input_dict = self.tokenizer(
            [text_input],
            add_special_tokens=True,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
        )
        input_dict.to(model_device)
        predictions = self.model(**input_dict).logits

        try:
            labels = predictions.argmax(dim=1)
            loss = self.model(**input_dict, labels=labels)[0]
        except TypeError:
            raise TypeError(
                f"{type(self.model)} class does not take in `labels` to calculate loss. "
                "One cause for this might be if you instantiatedyour model using `transformer.AutoModel` "
                "(instead of `transformers.AutoModelForSequenceClassification`)."
            )

        loss.backward()

        # grad w.r.t to word embeddings
        grad = emb_grads[0][0].cpu().numpy()

        embedding_layer.weight.requires_grad = original_state
        emb_hook.remove()
        self.model.eval()

        output = {"ids": input_dict["input_ids"], "gradient": grad}

        return output

    def _tokenize(self, inputs):
        """Helper method that for `tokenize`
        Args:
            inputs (list[str]): list of input strings
        Returns:
            tokens (list[list[str]]): List of list of tokens as strings
        """
        return [
            self.tokenizer.convert_ids_to_tokens(
                self.tokenizer([x], truncation=True)["input_ids"][0]
            )
            for x in inputs
        ]

In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-ag-news")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/roberta-base-ag-news")
model_wrapper = MyWrapperVulnerableWordMask(model, tokenizer)

# Load the dataset
dataset = textattack.datasets.HuggingFaceDataset("ag_news", split="test")

In [ ]:
# recipe: textfooler

# build the textfooler attack
attack = textattack.attack_recipes.textfooler_jin_2019.TextFoolerJin2019.build(model_wrapper)

# run the attack
attack_args = textattack.AttackArgs(
    num_examples=10,
    log_to_csv="1_textfooler_20percent_vulnerable_words.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=True
)

attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:10<00:00, 47.4MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpc2g1yrjc.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path 1_textfooler_20percent_vulnerable_words.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:  50%|█████     | 5/10 [14:03<14:03, 168.75s/it]textattack: Saving checkpoint under "checkpoints/1679754482757.ta.chkpt" at 2023-03-25 14:28:02 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 2 / 0 / 10: 100%|██████████| 10/10 [1:28:03<00:00, 528.37s/it]textattack: Saving checkpoint under "checkpoints/1679758922751.ta.chkpt" at 2023-03-25 15:42:02 after 10 attacks.
[Succeeded / Failed / Skipped / Total] 8 / 2 / 0 / 10: 100%|██████████| 10/10 [1:28:03<00:00, 528.37s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 80.0%  |
| Average perturbed word %:     | 17.18% |
| Average num. words per input: | 63.0   |
| Avg num queries:              | 488.7  |
+-------------------------------+--------+


In [ ]:
# recipe: textfooler

# build the textfooler attack
attack = textattack.attack_recipes.textfooler_jin_2019.TextFoolerJin2019.build(model_wrapper)

# run the attack
attack_args = textattack.AttackArgs(
    num_examples=50,
    log_to_csv="textfooler_20percent_vulnerable_words.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=True
)

attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

In [ ]:
# recipe: BERT attack

# build the textfooler attack
attack = textattack.attack_recipes.bert_attack_li_2020.BERTAttackLi2020.build(model_wrapper)

# run the attack
attack_args = textattack.AttackArgs(
    num_examples=10,
    log_to_csv="1_BERTattack_20percent_vulnerable_words.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=True
)

attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path 1_BERTattack_20percent_vulnerable_words.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bert-attack
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  48
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.4
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.2
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:  50%|█████     | 5/10 [53:39<53:39, 643.85s/it]textattack: Saving checkpoint under "checkpoints/1679762148093.ta.chkpt" at 2023-03-25 16:35:48 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 0 / 0 / 6:  60%|██████    | 6/10 [1:51:30<1:14:20, 1115.01s/it]

### other stuff

In [ ]:
# what's the mask token?
tokenizer.mask_token

'<mask>'

In [ ]:
# Tokenize the input sentence and get the corresponding tensor
def tokenize(sentence):
    return tokenizer(sentence,
                     add_special_tokens=True,
                     padding=True,
                     truncation=True,
                     return_tensors="pt")

def mask_combinations(words):
    combinations = []
    for i in range(len(words)):
        masked_sentence = " ".join(["<mask>" if k == i else words[k] for k in range(len(words))])
        combinations.append(masked_sentence)
    return combinations

# Get the importance score of each word in the input sentence
def get_importance_score(sentence):
    words = sentence.split()
    combinations = mask_combinations(words)
    scores = []
    with torch.no_grad():
        original_logit = model(tokenize(sentence)["input_ids"])[0]
    for combination in combinations:
        input_ids = tokenize(combination)["input_ids"]
        with torch.no_grad():
            outputs = model(input_ids)
            # get the euclidean distance between original logit and the result
            score = torch.dist(original_logit, outputs[0])
        scores.append(score)
    return scores

In [ ]:
bert_attack = pd.read_csv("log_BERTattack.csv")

In [47]:
original = bert_attack['original_text']
perturbed = bert_attack['perturbed_text']

In [ ]:
importance = get_importance_score(original)

In [ ]:
np.array([float(t) for t in importance])

array([0.02197488, 0.03018123, 0.01974173, 0.15918078, 0.01023945,
       0.06014449, 0.04826671, 0.07275195, 0.07217523, 0.02086288,
       0.04624876, 0.02653462, 0.04765706, 0.39498734, 0.50361991,
       0.09728913, 0.09184477, 0.28331938, 0.03203112, 0.08427735,
       0.0804624 , 0.02171165, 0.1073916 , 0.09420253, 0.05850932,
       0.10105842, 0.05590782, 0.10888957, 0.16191019, 0.04632794,
       0.20375611, 0.04212094, 0.02963632, 0.22915542, 0.0465832 ,
       0.24372447, 0.04867834, 0.07482402, 0.05349797, 0.02644884,
       0.06549034, 0.01860557, 0.10122984, 0.03046717, 0.05135306,
       0.10009118, 0.0377837 , 0.22190085])

In [ ]:
# create a list of tuples where each tuple contains a word and its corresponding importance score
original_importance_list = [(word, score) for word, score in zip(original.split(), get_importance_score(original))]
perturbed_importance_list = [(word, score) for word, score in zip(perturbed.split(), get_importance_score(perturbed))]

In [43]:
def compare_ranks(sen1, sen2):
    original = np.array([float(t) for t in get_importance_score(sen1)])
    perturbed = np.array([float(t) for t in get_importance_score(sen2)])
    
    # get the ranking for both
    sorted_indices = original.argsort()
    original_ranks = np.empty_like(sorted_indices)
    original_ranks[sorted_indices] = np.arange(len(original)) + 1

    sorted_indices = perturbed.argsort()
    perturbed_ranks = np.empty_like(sorted_indices)
    perturbed_ranks[sorted_indices] = np.arange(len(perturbed)) + 1

    count = 0
    for i in range(len(perturbed_ranks)):
        if original_ranks[i] == perturbed_ranks[i]:
            count += 1
    # return the ratio of same words with same ranking 
    return count/len(original_ranks)

def top3_in_top6(sen1, sen2):
    original = np.array([float(t) for t in get_importance_score(sen1)])
    perturbed = np.array([float(t) for t in get_importance_score(sen2)])
    
    # get the ranking for both
    sorted_indices = original.argsort()
    original_ranks = np.empty_like(sorted_indices)
    original_ranks[sorted_indices] = np.arange(len(original)) + 1

    sorted_indices = perturbed.argsort()
    perturbed_ranks = np.empty_like(sorted_indices)
    perturbed_ranks[sorted_indices] = np.arange(len(perturbed)) + 1

    # Find the top 3 indices in original_ranks
    top3_indices_original_ranks = sorted(range(len(original_ranks)), key=lambda i: original_ranks[i])[:3]

    # Check if the indices are in the top 6 of perturbed_ranks (boolean)
    return all(perturbed_ranks[idx] <= 6 for idx in top3_indices_original_ranks)

In [30]:
similarity = []
for i in range(len(original)):
    similarity.append(compare_ranks(original[i], perturbed[i]))

In [31]:
similarity

[0.16,
 0.020833333333333332,
 0.044444444444444446,
 0.01818181818181818,
 0.08823529411764706]

In [4]:
sum([0.16,
 0.020833333333333332,
 0.044444444444444446,
 0.01818181818181818,
 0.08823529411764706])/5

0.06633897801544861

In [48]:
top3INtop6 = []
for i in range(len(original)):
    top3INtop6.append(top3_in_top6(original[i], perturbed[i]))
top3INtop6

[False, False, False, False, False]

In [ ]:
# from BERT attack github
filter_words = ['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'ain', 'all', 'almost',
                'alone', 'along', 'already', 'also', 'although', 'am', 'among', 'amongst', 'an', 'and', 'another',
                'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'aren', "aren't", 'around', 'as',
                'at', 'back', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides',
                'between', 'beyond', 'both', 'but', 'by', 'can', 'cannot', 'could', 'couldn', "couldn't", 'd', 'didn',
                "didn't", 'doesn', "doesn't", 'don', "don't", 'down', 'due', 'during', 'either', 'else', 'elsewhere',
                'empty', 'enough', 'even', 'ever', 'everyone', 'everything', 'everywhere', 'except', 'first', 'for',
                'former', 'formerly', 'from', 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'he', 'hence',
                'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his',
                'how', 'however', 'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'isn', "isn't", 'it', "it's",
                'its', 'itself', 'just', 'latter', 'latterly', 'least', 'll', 'may', 'me', 'meanwhile', 'mightn',
                "mightn't", 'mine', 'more', 'moreover', 'most', 'mostly', 'must', 'mustn', "mustn't", 'my', 'myself',
                'namely', 'needn', "needn't", 'neither', 'never', 'nevertheless', 'next', 'no', 'nobody', 'none',
                'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'o', 'of', 'off', 'on', 'once', 'one', 'only',
                'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'per',
                'please', 's', 'same', 'shan', "shan't", 'she', "she's", "should've", 'shouldn', "shouldn't", 'somehow',
                'something', 'sometime', 'somewhere', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs',
                'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein',
                'thereupon', 'these', 'they', 'this', 'those', 'through', 'throughout', 'thru', 'thus', 'to', 'too',
                'toward', 'towards', 'under', 'unless', 'until', 'up', 'upon', 'used', 've', 'was', 'wasn', "wasn't",
                'we', 'were', 'weren', "weren't", 'what', 'whatever', 'when', 'whence', 'whenever', 'where',
                'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while',
                'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'with', 'within', 'without', 'won',
                "won't", 'would', 'wouldn', "wouldn't", 'y', 'yet', 'you', "you'd", "you'll", "you're", "you've",
                'your', 'yours', 'yourself', 'yourselves']

def top_n_words(word_importance_list, n):

    filtered_list = [pair for pair in word_importance_list if pair[0] not in filter_words]
    
    # Sort the list in descending order based on the importance value
    sorted_list = sorted(filtered_list, key=lambda x: x[1], reverse=True)
    
    # Get the top n vulnerable words
    top_n_words = [word for word, _ in sorted_list[:n]]
    
    return top_n_words



In [ ]:
original

'The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\\team of rocketeers competing for the  #36;10 million Ansari X Prize, a contest for\\privately funded suborbital space flight, has officially announced the first\\launch date for its manned rocket.'

In [ ]:
top_n_words(original_importance_list, 10)

['SPACE.com',
 '(SPACE.com)',
 'Canada',
 'suborbital',
 'for\\privately',
 'rocket.',
 'Prize,',
 'Ansari',
 'On:',
 'million']

In [ ]:
perturbed

'The Racing is Around: Second Privately Group Sets Opened Times for Humane Spaceflight (SPACE.com) SATELLITE.kom - TORONTO, Montreal -- A second\\machines of rocketeers competes for the  #36;10 billion Ansari X Prix, a defiant for\\privately subsidies suborbital margin larceny, hectares formally advert the first\\startup period for its desolate bombardments.'

In [ ]:
top_n_words(perturbed_importance_list, 10)

['Spaceflight',
 'bombardments.',
 'TORONTO,',
 '(SPACE.com)',
 'Racing',
 'rocketeers',
 'Privately',
 'Humane',
 'for\\privately',
 'Montreal']

also check for textfooler

In [49]:
textfooler = pd.read_csv("log_textfooler.csv")
original = textfooler['original_text'][:100]
perturbed = textfooler['perturbed_text'][:100]

In [34]:
similarity = []
for i in range(len(original)):
    similarity.append(compare_ranks(original[i], perturbed[i]))

In [35]:
similarity

[0.08,
 0.020833333333333332,
 0.044444444444444446,
 0.07272727272727272,
 0.029411764705882353,
 0.014705882352941176,
 0.038834951456310676,
 0.02654867256637168,
 0.058823529411764705,
 0.0967741935483871,
 0.020833333333333332,
 0.023809523809523808,
 0.023255813953488372,
 0.043478260869565216,
 0.2777777777777778,
 0.0,
 0.024390243902439025,
 0.0,
 0.044444444444444446,
 0.041666666666666664,
 0.08333333333333333,
 0.05555555555555555,
 0.058823529411764705,
 1.0,
 0.05,
 0.12,
 0.03225806451612903,
 0.014925373134328358,
 0.02857142857142857,
 0.014492753623188406,
 0.08571428571428572,
 0.0,
 0.09090909090909091,
 0.057692307692307696,
 0.05454545454545454,
 0.015625,
 0.08333333333333333,
 0.05,
 0.0,
 0.021739130434782608,
 0.02631578947368421,
 0.04878048780487805,
 0.08163265306122448,
 0.10638297872340426,
 0.0392156862745098,
 0.12903225806451613,
 0.05405405405405406,
 0.04,
 0.0,
 0.06060606060606061,
 0.0425531914893617,
 0.020833333333333332,
 0.025,
 0.0,
 0.0,
 0.

In [37]:
np.mean(similarity)

0.06272142922450179

In [ ]:
top3INtop6 = []
for i in range(len(original)):
    top3INtop6.append(top3_in_top6(original[i], perturbed[i]))

In [53]:
sum(top3INtop6)/len(top3INtop6)

0.02